In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

path_root = '../..'
sys.path.append(f'{path_root}/')
from src.util.functions import Functions as F

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = f'{path_root}/data'
path_csv = f'{path_data}/csv'

csv_out_name = f'{path_csv}/covid19-{level}.csv'
pop_regions_name = f'{path_csv}/popolazione-regioni.csv'
icu_regions_name = f'{path_csv}/terapie-intensive.csv'
regions_name = f'{path_csv}/regioni.csv'

In [2]:
url_cum = F.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,note,ingressi_terapia_intensiva,note_test,note_casi,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14275,2022-01-03T17:00:00,ITA,19,Sicilia,38.115697,13.362357,872,112,984,54396,...,DECEDUTI: N. 1 IL 03/01/22 - N. 3 IL 02/01/22 ...,11.0,NaN,NaN,386716.0,0.0,3868203.0,4642964.0,ITG,ITG1
14276,2022-01-03T17:00:00,ITA,9,Toscana,43.769231,11.255889,755,84,839,99944,...,NaN,5.0,NaN,NaN,375401.0,34511.0,5504933.0,3749789.0,ITI,ITI1
14277,2022-01-03T17:00:00,ITA,10,Umbria,43.106758,12.388247,162,10,172,24233,...,NaN,3.0,NaN,NaN,85057.0,9545.0,1357822.0,1577134.0,ITI,ITI2
14278,2022-01-03T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,45,3,48,2783,...,NaN,0.0,NaN,NaN,13759.0,3228.0,126657.0,237103.0,ITC,ITC2


In [4]:
list(df_raw_r.columns)

['data',
 'stato',
 'codice_regione',
 'denominazione_regione',
 'lat',
 'long',
 'ricoverati_con_sintomi',
 'terapia_intensiva',
 'totale_ospedalizzati',
 'isolamento_domiciliare',
 'totale_positivi',
 'variazione_totale_positivi',
 'nuovi_positivi',
 'dimessi_guariti',
 'deceduti',
 'casi_da_sospetto_diagnostico',
 'casi_da_screening',
 'totale_casi',
 'tamponi',
 'casi_testati',
 'note',
 'ingressi_terapia_intensiva',
 'note_test',
 'note_casi',
 'totale_positivi_test_molecolare',
 'totale_positivi_test_antigenico_rapido',
 'tamponi_test_molecolare',
 'tamponi_test_antigenico_rapido',
 'codice_nuts_1',
 'codice_nuts_2']

In [5]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'casi_da_sospetto_diagnostico', 'casi_da_screening', 'note', 'note_test', 'note_casi']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,...,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,...,0,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,...,18,148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14275,2022-01-03T17:00:00,19,Sicilia,872,112,984,54396,55380,4084,4384,...,386716,8511167,3060865.0,11.0,386716.0,0.0,3868203.0,4642964.0,ITG,ITG1
14276,2022-01-03T17:00:00,9,Toscana,755,84,839,99944,100783,5986,6952,...,409912,9254722,3701426.0,5.0,375401.0,34511.0,5504933.0,3749789.0,ITI,ITI1
14277,2022-01-03T17:00:00,10,Umbria,162,10,172,24233,24405,1042,1134,...,94602,2934956,540810.0,3.0,85057.0,9545.0,1357822.0,1577134.0,ITI,ITI2
14278,2022-01-03T17:00:00,2,Valle d'Aosta,45,3,48,2783,2831,238,296,...,16987,363760,112311.0,0.0,13759.0,3228.0,126657.0,237103.0,ITC,ITC2


In [6]:
# Compute most recent couple of dates
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = F.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2022-01-03', '2022-01-02')

In [7]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2022-01-03T17:00:00,1,Piemonte,1354,112,1466,81995,83461,3370,4583,...,509421,11462225,3107394.0,8.0,403948.0,105473.0,4178219.0,7284006.0,ITC,ITC1
1,2022-01-03T17:00:00,2,Valle d'Aosta,45,3,48,2783,2831,238,296,...,16987,363760,112311.0,0.0,13759.0,3228.0,126657.0,237103.0,ITC,ITC2
2,2022-01-03T17:00:00,3,Lombardia,2147,219,2366,301496,303862,11769,13421,...,1278480,25250696,6880868.0,18.0,999907.0,278573.0,13119014.0,12131682.0,ITC,ITC4
3,2022-01-03T17:00:00,5,Veneto,1170,186,1356,110233,111589,4790,6468,...,670277,19808236,2476833.0,6.0,596679.0,73598.0,8349672.0,11458564.0,ITH,ITH3
4,2022-01-03T17:00:00,6,Friuli Venezia Giulia,308,30,338,14467,14805,39,453,...,160056,4563571,970015.0,3.0,138418.0,21638.0,2663418.0,1900153.0,ITH,ITH4
5,2022-01-03T17:00:00,7,Liguria,532,46,578,13196,13774,-85,815,...,153572,3581938,1053669.0,4.0,153572.0,0.0,2009655.0,1572283.0,ITC,ITC3
6,2022-01-03T17:00:00,8,Emilia-Romagna,1528,131,1659,102167,103826,6774,8014,...,566261,11769169,2411804.0,8.0,553872.0,12389.0,7346025.0,4423144.0,ITH,ITH5
7,2022-01-03T17:00:00,9,Toscana,755,84,839,99944,100783,5986,6952,...,409912,9254722,3701426.0,5.0,375401.0,34511.0,5504933.0,3749789.0,ITI,ITI1
8,2022-01-03T17:00:00,10,Umbria,162,10,172,24233,24405,1042,1134,...,94602,2934956,540810.0,3.0,85057.0,9545.0,1357822.0,1577134.0,ITI,ITI2
9,2022-01-03T17:00:00,11,Marche,232,54,286,7853,8139,30,519,...,148301,2034762,1197760.0,4.0,148301.0,0.0,1656906.0,377856.0,ITI,ITI3


In [8]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2022-01-02T17:00:00,1,Piemonte,1294,109,1403,78688,80091,1551,2619,...,504838,11421200,3100684.0,6.0,403025.0,101813.0,4172478.0,7248722.0,ITC,ITC1
1,2022-01-02T17:00:00,2,Valle d'Aosta,36,3,39,2554,2593,114,117,...,16691,361884,111580.0,0.0,13624.0,3067.0,126217.0,235667.0,ITC,ITC2
2,2022-01-02T17:00:00,3,Lombardia,2052,216,2268,289825,292093,7196,10425,...,1265059,25183247,6868839.0,15.0,993570.0,271489.0,13094013.0,12089234.0,ITC,ITC4
3,2022-01-02T17:00:00,5,Veneto,1120,188,1308,105491,106799,2908,3816,...,663809,19758438,2472640.0,16.0,592987.0,70822.0,8333719.0,11424719.0,ITH,ITH3
4,2022-01-02T17:00:00,6,Friuli Venezia Giulia,298,28,326,14440,14766,918,1188,...,159605,4556503,969305.0,4.0,138151.0,21454.0,2661326.0,1895177.0,ITH,ITH4
5,2022-01-02T17:00:00,7,Liguria,510,47,557,13302,13859,427,1265,...,152757,3573911,1052169.0,3.0,152757.0,0.0,2006357.0,1567554.0,ITC,ITC3
6,2022-01-02T17:00:00,8,Emilia-Romagna,1451,130,1581,95471,97052,8186,9090,...,558263,11738029,2407107.0,10.0,547665.0,10598.0,7329428.0,4408601.0,ITH,ITH5
7,2022-01-02T17:00:00,9,Toscana,695,85,780,94017,94797,5921,6367,...,402960,9227083,3689948.0,9.0,370906.0,32054.0,5489260.0,3737823.0,ITI,ITI1
8,2022-01-02T17:00:00,10,Umbria,151,8,159,23204,23363,539,788,...,93468,2926140,538529.0,1.0,84524.0,8944.0,1355436.0,1570704.0,ITI,ITI2
9,2022-01-02T17:00:00,11,Marche,227,50,277,7832,8109,348,699,...,147782,2030264,1194317.0,4.0,147782.0,0.0,1654957.0,375307.0,ITI,ITI3


In [9]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = df_raw_r_1
# df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
# df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

In [10]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = df_raw_r_0
# df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
# df_r_0.sort_values(by='codice_regione').reset_index()

In [11]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione
0,2022-01-03T17:00:00,1,Piemonte,1354,112,1466,81995,83461,3370,4583,...,11462225,3107394.0,8.0,403948.0,105473.0,4178219.0,7284006.0,ITC,ITC1,4.341375e+06
1,2022-01-03T17:00:00,2,Valle d'Aosta,45,3,48,2783,2831,238,296,...,363760,112311.0,0.0,13759.0,3228.0,126657.0,237103.0,ITC,ITC2,1.255010e+05
2,2022-01-03T17:00:00,3,Lombardia,2147,219,2366,301496,303862,11769,13421,...,25250696,6880868.0,18.0,999907.0,278573.0,13119014.0,12131682.0,ITC,ITC4,1.010397e+07
3,2022-01-03T17:00:00,5,Veneto,1170,186,1356,110233,111589,4790,6468,...,19808236,2476833.0,6.0,596679.0,73598.0,8349672.0,11458564.0,ITH,ITH3,4.907704e+06
4,2022-01-03T17:00:00,6,Friuli Venezia Giulia,308,30,338,14467,14805,39,453,...,4563571,970015.0,3.0,138418.0,21638.0,2663418.0,1900153.0,ITH,ITH4,1.211357e+06
5,2022-01-03T17:00:00,7,Liguria,532,46,578,13196,13774,-85,815,...,3581938,1053669.0,4.0,153572.0,0.0,2009655.0,1572283.0,ITC,ITC3,1.543127e+06
6,2022-01-03T17:00:00,8,Emilia-Romagna,1528,131,1659,102167,103826,6774,8014,...,11769169,2411804.0,8.0,553872.0,12389.0,7346025.0,4423144.0,ITH,ITH5,4.467118e+06
7,2022-01-03T17:00:00,9,Toscana,755,84,839,99944,100783,5986,6952,...,9254722,3701426.0,5.0,375401.0,34511.0,5504933.0,3749789.0,ITI,ITI1,3.722729e+06
8,2022-01-03T17:00:00,10,Umbria,162,10,172,24233,24405,1042,1134,...,2934956,540810.0,3.0,85057.0,9545.0,1357822.0,1577134.0,ITI,ITI2,8.802850e+05
9,2022-01-03T17:00:00,11,Marche,232,54,286,7853,8139,30,519,...,2034762,1197760.0,4.0,148301.0,0.0,1656906.0,377856.0,ITI,ITI3,1.518400e+06


In [12]:
# Update icu csv data
df_icu_updated_r = pd.DataFrame.from_dict(F.icu_data())
df_icu_updated_r.to_csv(icu_regions_name, index=False)

In [13]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, left_on='denominazione_regione', right_on='regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione,regione,ricoverati_area_non_critica,posti_letto_area_non_critica,ricoverati_terapia_intensiva,posti_letto_terapia_intensiva,posti_letto_terapia_intensiva_attivabili
0,2022-01-03T17:00:00,1,Piemonte,1354,112,1466,81995,83461,3370,4583,...,7284006.0,ITC,ITC1,4.341375e+06,Piemonte,1354,5824,112,628,99
1,2022-01-03T17:00:00,2,Valle d'Aosta,45,3,48,2783,2831,238,296,...,237103.0,ITC,ITC2,1.255010e+05,Valle d'Aosta,45,99,3,33,0
2,2022-01-03T17:00:00,3,Lombardia,2147,219,2366,301496,303862,11769,13421,...,12131682.0,ITC,ITC4,1.010397e+07,Lombardia,2147,10457,219,1530,0
3,2022-01-03T17:00:00,5,Veneto,1170,186,1356,110233,111589,4790,6468,...,11458564.0,ITH,ITH3,4.907704e+06,Veneto,1170,6000,186,1000,0
4,2022-01-03T17:00:00,6,Friuli Venezia Giulia,308,30,338,14467,14805,39,453,...,1900153.0,ITH,ITH4,1.211357e+06,Friuli Venezia Giulia,308,1277,30,175,0
5,2022-01-03T17:00:00,7,Liguria,532,46,578,13196,13774,-85,815,...,1572283.0,ITC,ITC3,1.543127e+06,Liguria,532,1764,46,219,12
6,2022-01-03T17:00:00,8,Emilia-Romagna,1528,131,1659,102167,103826,6774,8014,...,4423144.0,ITH,ITH5,4.467118e+06,Emilia-Romagna,1528,9001,131,889,0
7,2022-01-03T17:00:00,9,Toscana,755,84,839,99944,100783,5986,6952,...,3749789.0,ITI,ITI1,3.722729e+06,Toscana,755,5033,84,570,25
8,2022-01-03T17:00:00,10,Umbria,162,10,172,24233,24405,1042,1134,...,1577134.0,ITI,ITI2,8.802850e+05,Umbria,162,662,10,86,41
9,2022-01-03T17:00:00,11,Marche,232,54,286,7853,8139,30,519,...,377856.0,ITI,ITI3,1.518400e+06,Marche,232,995,54,252,53


In [14]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [15]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['posti_letto_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['posti_letto_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2022-01-03T17:00:00,3,Lombardia,2147,219,2366,301496,303862,11769,13421,...,14.31,11671,2996,67449,12029.0,19,1633,19.90,0.13283,12.65
1,2022-01-03T17:00:00,8,Emilia-Romagna,1528,131,1659,102167,103826,6774,8014,...,14.74,6696,-1076,31140,4697.0,17,1207,25.74,0.17940,12.68
2,2022-01-03T17:00:00,9,Toscana,755,84,839,99944,100783,5986,6952,...,14.74,5927,585,27639,11478.0,6,960,25.15,0.18674,11.01
3,2022-01-03T17:00:00,15,Campania,750,60,810,98629,99439,5061,6653,...,8.94,5024,1461,48768,6750.0,21,1571,13.64,0.11499,10.53
4,2022-01-03T17:00:00,5,Veneto,1170,186,1356,110233,111589,4790,6468,...,18.60,4742,2652,49798,4193.0,9,1669,12.99,0.13179,13.66
5,2022-01-03T17:00:00,12,Lazio,1199,158,1357,93409,94766,4365,5614,...,16.76,4333,-2379,43902,3282.0,15,1234,12.79,0.09571,9.06
6,2022-01-03T17:00:00,1,Piemonte,1354,112,1466,81995,83461,3370,4583,...,17.83,3307,1964,41025,6710.0,7,1206,11.17,0.10557,11.73
7,2022-01-03T17:00:00,19,Sicilia,872,112,984,54396,55380,4084,4384,...,13.13,4018,420,25286,10644.0,16,284,17.34,0.08824,7.78
8,2022-01-03T17:00:00,16,Puglia,299,33,332,33644,33976,3298,3568,...,6.69,3281,117,30923,6122.0,1,269,11.54,0.08902,7.96
9,2022-01-03T17:00:00,13,Abruzzo,194,22,216,23487,23703,1703,1990,...,12.15,1693,855,15691,2370.0,3,284,12.68,0.15240,8.71


In [16]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [17]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'posti_letto_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,posti_letto_terapia_intensiva,saturazione_terapia_intensiva
0,P.A. Trento,22,68,90,24.44
1,Marche,54,198,252,21.43
2,Liguria,46,173,219,21.00
3,Veneto,186,814,1000,18.60
4,P.A. Bolzano,18,82,100,18.00
5,Piemonte,112,516,628,17.83
6,Friuli Venezia Giulia,30,145,175,17.14
7,Lazio,158,785,943,16.76
8,Calabria,29,160,189,15.34
9,Toscana,84,486,570,14.74


In [18]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [19]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../../data/csv/regioni/covid19-veneto.csv
Aggiornato ../../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../../data/csv/regioni/covid19-liguria.csv
Aggiornato ../../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../../data/csv/regioni/covid19-toscana.csv
Aggiornato ../../data/csv/regioni/covid19-umbria.csv
Aggiornato ../../data/csv/regioni/covid19-marche.csv
Aggiornato ../../data/csv/regioni/covid19-lazio.csv
Aggiornato ../../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../../data/csv/regioni/covid19-molise.csv
Aggiornato ../../data/csv/regioni/covid19-campania.csv
Aggiornato ../../data/csv/regioni/covid19-puglia.csv
Aggiornato ../../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../../data/csv/regioni/covid19-calabria.csv
Aggiornato ../../data/csv/regioni/covid19-sicilia.csv


In [20]:
# df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
# df_tb

In [21]:
# Initialize df_taa
# df_taa = Functions.replace_bt_with_taa(df_tb[:2])
# for i in range(2, df_tb.shape[0], 2):
#     df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
# df_taa = add_cols(df_taa)
# df_taa

In [22]:
# region_name = df_taa.denominazione_regione.iloc[0].lower()
# csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
# df_taa.to_csv(csv_out_region_name, index=False)
# print('Aggiornato {file}'.format(file=csv_out_region_name))